In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get
import re

# Pulling all links off of the search pages (up to 3000) and putting them in a dataframe to scrape.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Machine+Learning+Engineer%22&l=Pittsburgh%2C+PA&start={}"
max_results=250
Linkdf=[]

for start in range(0, max_results, 7):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

324

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])

#Sanity Check
#print(websites)
len(websites)

324

In [5]:
websites1=set(websites)

In [6]:
len(websites1)

9

# Looping through websites....

In [7]:
Title=[]
Descriptions=[]
Location=[]
FullDescriptions=[]

for url in websites1:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    try:
        locations=str(location_containers.text)
    except AttributeError:
        locations = 'None Found'
    try:
        full_descriptions = str(description_containers.text)
    except AttributeError:
        full_descriptions= 'None Found'
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)
    FullDescriptions.append(full_descriptions)    

# Extracting what we want from the Descriptions Column

In [8]:
#'Location' left in for sanity check. Should be removed once code is confirmed to work
Descriptions_df = pd.DataFrame(columns = ['Title', 'Location','City', 'State', 'Zip', 'Country', 'Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement', 'FullDescriptions'])
Country = ['US', 'USA', 'United States', 'United States of Americal']
States = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
          'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
          'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

for index, element in enumerate(Descriptions):
    soup=BeautifulSoup(element,'lxml')
    for values in list(Descriptions_df):
        temp_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = temp_tag.find_next('ul')
            Descriptions_df.at[index,values] = ul_tag.text
        except AttributeError:
            Descriptions_df.at[index,values]="None Found"
        Descriptions_df.at[index,"Title"]=Title[index]
        Descriptions_df.at[index,"Location"]=Location[index]
        Descriptions_df.at[index,"FullDescriptions"]=FullDescriptions[index]
        words = '|'.join(Country)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Country"] = temp[0]
        words = '|'.join(States)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"State"] = temp[0]
        temp = re.findall(r'\d+', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Zip"] = temp[0]  
            
        temp = re.findall(r'[\w w]+,', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"City"] = re.sub(',', '', temp[0])


In [9]:
Descriptions_df

Title  \
0  Machine Learning Engineer                                                
1  DATA SCIENTIST / MACHINE LEARNING ENGINEER                               
2  Senior Machine Learning Engineer                                         
3  Machine Learning Engineer, National Robotics Engineering Center (NREC)   
4  Sr Machine Learning Engineer - Language & Speech Processing              
5  Machine Learning Engineer                                                
6  Machine Learning Engineer - Autonomous Driving                           
7  Machine Learning Engineer                                                
8  Embedded Machine Learning Engineer                                       

               Location        City State         Zip     Country  \
0  Pittsburgh, PA        Pittsburgh  PA    None Found  None Found   
1  Pittsburgh, PA        Pittsburgh  PA    None Found  None Found   
2  Pittsburgh, PA        Pittsburgh  PA    None Found  None Found   
3  Pittsburgh, PA        Pittsburgh  PA    None Found  None Found   
4  Pittsburgh, PA        Pittsburgh  PA    None Found  None Found   
5  Pittsburgh, PA 15237  Pittsburgh  PA    15237       None Found   
6  Pittsburgh, PA 15238  Pittsburgh  PA    15238       None Found   
7  Pittsburgh, PA 15206  Pittsburgh  PA    15206       None Found   
8  Pittsburgh, PA 15222  Pittsburgh  PA    15222       None Found   

                                                                                                                                                                                                                                                                                                                                                                                                     Qualifications  \
0  None Found                                                                                                                                                                                                                                                                                                                                                                                                         
1  None Found                                                                                                                                                                                                                                                                                                                                                                                                         
2  None Found                                                                                                                                                                                                                                                                                                                                                                                                         
3  \nB.S. in Computer Science, Engineering, Mathematics is required (More is a bonus)\n1-3 years combined graduate academic and professional software development experience is required\nExperience applying machine learning algorithms\nExperience with scripting languages for quick prototyping (Python, Matlab preferred)\nKnowledge of basic software engineering principles\nTechnical communication skills   
4  None Found                                                                                                                                                                                                                                                                                                                                                                                                         
5  None Found                                                                                                                                                       

# Move to csv

In [10]:
Descriptions_df.to_csv('Descriptions_df_MLE_Pittsburgh.csv')